In [432]:
!pip install -U tfds-nightly --user

     |████████████████████████████████| 3.5 MB 4.6 MB/s eta 0:00:01
  Attempting uninstall: tfds-nightly
    Found existing installation: tfds-nightly 3.2.1.dev202008180105
    Uninstalling tfds-nightly-3.2.1.dev202008180105:
      Successfully uninstalled tfds-nightly-3.2.1.dev202008180105
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [433]:
!pip install tf_slim

In [434]:
!pip install pycocotools

In [435]:
import numpy as np
import os
import six
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display



In [436]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [437]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto:5:1: warning: Import object_detection/protos/image_resizer.proto is unused.


In [438]:
%%bash 
cd models/research
pip install .

Processing /home/jupyter/hazard-detection/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1310302 sha256=ff4ffe6c8f5b479c8162411aaf9539247bc3f6923b6bede63cdecf3cf6b9cc3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-34ngkkzj/wheels/2f/69/68/d174ba7c18fe52ba5ad48455b812a18ad3d77a17bed688d8be
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [439]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [440]:
import tensorflow_datasets as tfds


In [441]:
!gcloud config set account application-default

Updated property [core/account].


In [442]:
(ds_train, ds_test), ds_info = tfds.load(
    'kitti',
    split=['train', 'test'],
    with_info=True,
    download=False,
#     data_dir='./tensorflow_datasets'
    data_dir="gs://kitti-dataset-1"
)


In [443]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [444]:

model_name = 'faster_rcnn_resnet50_lowproposals_coco_2018_01_28'
# model_name = 'faster_rcnn_resnet101_lowproposals_coco_2018_01_28'

detection_model = load_model(model_name)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [445]:
# List of the strings that is used to add correct label for each box.

# PATH_TO_LABELS = 'models/research/object_detection/data/kitti_label_map.pbtxt'
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


In [447]:
ds_train = ds_train.cache()
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [448]:
ds_train

<PrefetchDataset shapes: {image: (None, None, 3), image/file_name: (), objects: {alpha: (None,), bbox: (None, 4), dimensions: (None, 3), location: (None, 3), occluded: (None,), rotation_y: (None,), truncated: (None,), type: (None,)}}, types: {image: tf.uint8, image/file_name: tf.string, objects: {alpha: tf.float32, bbox: tf.float32, dimensions: tf.float32, location: tf.float32, occluded: tf.int64, rotation_y: tf.float32, truncated: tf.float32, type: tf.int64}}>

In [449]:

ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [450]:
print(detection_model.inputs)


[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [451]:
detection_model.output_dtypes

{'num_detections': tf.float32,
 'detection_boxes': tf.float32,
 'detection_classes': tf.float32,
 'detection_scores': tf.float32}

In [452]:
detection_model.output_shapes

{'num_detections': TensorShape([None]),
 'detection_boxes': TensorShape([None, 20, 4]),
 'detection_classes': TensorShape([None, 20]),
 'detection_scores': TensorShape([None, 20])}

## Output format


- Output classes are always integers in the range 0, num_classes). Any mapping of these integers to semantic labels is to be handled outside of this class. We never explicitly emit a “background class” --- thus 0 is the first non-background class and any logic of predicting and removing implicit background classes must be handled internally by the implementation.


- Detected boxes are to be interpreted as being in (y_min, x_min, y_max, x_max) format and normalized relative to the image window.

In [453]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

 bbox: RawBoundingBox, bounding box in Kitti coordinates (origin top left).

In [454]:
def norm(data):
    height= 375
    width = 1242
    
    data.loc[:,'xmin'] = data['xmin'] / width 
    data.loc[:,'xmax'] = data['xmax'] / width
    data.loc[:,'ymax'],data.loc[:,'ymin']  = (height - data['ymin']) / height , (height - data['ymax']) / height
    
    return data

In [455]:
MODEL = "model@1597671595"
WEIGHTS = "model@1597671595"


# load json and create model
json_file = open('generated_files/{}.json'.format(MODEL), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json( loaded_model_json )

# load weights into new model
loaded_model.load_weights("generated_files/{}.h5".format(WEIGHTS))
print("Loaded model from disk")

loaded_model.compile(loss='mean_squared_error', optimizer='adam')


Loaded model from disk


In [456]:

categories = [{
        'id': 1,
        'name': 'car'
    }, {
        'id': 2,
        'name': 'van'
    }, {
        'id': 3,
        'name': 'truck'
    }, {
        'id': 4,
        'name': 'pedestrian'
    }]

cat_index  = {i+1: val for i,val in enumerate(categories) }

In [457]:

def run_locnet( bboxes):
  
    if len(bboxes)==0 or len(bboxes)==1 and not any(bboxes[0]):
        return []
    y_pred = loaded_model.predict(bboxes)
  
    return np.hstack((bboxes,y_pred))


In [458]:
def transform_to_kitti(classes):
    
    hash = { 3:1 , 8:3, 1:4, 7:7 }
    return [ hash.get(classes[i], 8) for i in range(len(classes)) ]

In [459]:
def filter_detections( output_dict):
    
    output_dict['detection_classes'] = transform_to_kitti(output_dict['detection_classes'])
    
#     print(output_dict  )
    misc_ids = (8,9)
    
    scores = output_dict['detection_scores']
    classes = output_dict['detection_classes']
    
    size = len(classes)
    min_threshhold = 0.5
    output_dict['detection_boxes'] = np.array([ output_dict['detection_boxes'][i] for i in range(size) if scores[i] >= min_threshhold and classes[i] not in misc_ids  ])
    output_dict['detection_classes'] = np.array([ output_dict['detection_classes'][i] for i in range(size) if scores[i] >= min_threshhold and classes[i] not in misc_ids  ])
    output_dict['detection_scores'] = np.array([ output_dict['detection_scores'][i] for i in range(size) if scores[i] >= min_threshhold and classes[i] not in misc_ids  ])


    return output_dict

In [460]:
def scale_up_boxes( output_dict):
    
    output_dict['detection_boxes'] = output_dict['detection_boxes'] * 1000

    return output_dict

In [461]:
!pip install filterpy

In [462]:
# initialize Sort object
from sort import *
mot_tracker = Sort( max_age=8, iou_threshold=0.3)


In [463]:
def reorder_for_sort(array):

     return np.array( [array[1] ,array[0], array[3], array[2]])

In [464]:
def format_detections_for_mot(outputdict):
    
    detections = [ np.append( reorder_for_sort(outputdict['detection_boxes'][i]) , outputdict['detection_scores'][i])  for i in range( len(outputdict['detection_classes'])) ] 
    
    return np.asarray(detections)  if len(detections) else np.empty((0, 5))


In [465]:
def format_boxes( boxes):
    
    result = boxes.numpy()
    for box in result:
        box[0]=1-box[0]
        box[2]=1-box[2]
        box[0],box[2] = box[2], box[0]
    return tf.convert_to_tensor(result)

In [466]:
def get_tracked_color(label):
    
    return vis_util.STANDARD_COLORS[label %len(vis_util.STANDARD_COLORS)]

In [467]:
def length(x1,y1,x2,y2):
    return sqrt( (x2-x1)**2 + (y2-y1)**2)
    

In [468]:
import copy
import cv2
import PIL.Image as Image
import PIL.ImageColor as ImageColor
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont
from math import sqrt

def process_image(image, objects=None, groundtruth=None, mode=None):
  

    image_np = np.copy(image)
    
    output_dict = run_inference_for_single_image(detection_model, image_np)
    
    output_dict =filter_detections( output_dict)

    max_age=25
    global frame
    global warning_ids
    thickness=1
    if mode =='tracking':
        
        detections = format_detections_for_mot(output_dict)

        tracked_objects = mot_tracker.update(detections)
        live_trackers = (i for i in mot_tracker.trackers if i.id+1 in (int(j[4]) for j in tracked_objects) )
        
#         warning_ids =[]
        for kt in live_trackers:
            
            k = copy.deepcopy(kt)
            center = convert_bbox_to_z(k.get_state()[0]).T[0]

            for x in range(60):
                predicted = k.predict()

            xmin,ymin,xmax,ymax = predicted[0] 
#             track_id = k.id+1
#             label = str(track_id)
#             vis_util.draw_bounding_box_on_image_array(image_np, ymin, xmin, ymax, xmax, thickness=1, display_str_list=[label],color=get_tracked_color(track_id))

            predicted_center = convert_bbox_to_z(predicted[0]).T[0]
            
            if center[2] < 0.4:

                image = Image.fromarray(image_np)
                draw = ImageDraw.Draw(image)
                im_width, im_height = image.size


                (x1,y1) = center[:2]
                (x2,y2) = predicted_center[:2]

                (x1_absolute, x2_absolute, y1_absolute, y2_absolute) = (x1 * im_width, x2 * im_width,
                                      y1 * im_height, y2 * im_height)

                draw.line([(x1_absolute, y1_absolute), (x2_absolute, y2_absolute)],
                          width=3,
                          fill='red')

                np.copyto(image_np, np.array(image))
            
                #hazard detection
                if 0.2 < predicted_center[0] < 0.8 and 0.75 < predicted_center[1] and length(x1,y1,x2,y2) > sqrt(center[2])*0.6:
                    warning_ids.append( (k.id+1,frame+max_age))
            
            
        warning_ids = [i for i in warning_ids if i[1]>frame ] #Warnings should persit for max_age frames, filter expired warnings here
        
        for object in tracked_objects:
            xmin,ymin,xmax,ymax = object[:4]
            track_id = int(object[4])
#             n= track_id%40 +1
            n= track_id
            label = f'object{n:03}'
            
            color = get_tracked_color(track_id)
            
            if convert_bbox_to_z(object).T[0][2] < 0.4:
                if track_id in (i[0] for i in warning_ids ):
                    color = 'red'
                    label = 'WARNING!'
                    thickness=4
                vis_util.draw_bounding_box_on_image_array(image_np, ymin, xmin, ymax, xmax, thickness=thickness, display_str_list=[label],color=color)


    elif mode=='locations':
        locations = run_locnet( output_dict['detection_boxes'])

        i=0
        while i < len(objects['location']) and i < len(locations):
            ymin,xmin,ymax,xmax = locations[i][:4]
    #             print( objects['location'].numpy() , i)
            coords = locations[i][4:7:2]
            groundtruth_coords = objects['location'].numpy()[i][0:3:2]
            label = "predicted:(%.1f,%.1f), actual:(%.1f,%.1f)" % tuple(np.append(coords  , groundtruth_coords).tolist())
            vis_util.draw_bounding_box_on_image_array(image_np, ymin, xmin, ymax, xmax, thickness=1, display_str_list=[label],color='green')
            i+=1

            
    else:
        # Visualization of the results of a detection.

        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          cat_index,
          use_normalized_coordinates=True,
          line_thickness=2)


        if groundtruth:
            groundtruth_boxes = format_boxes(objects['bbox']).numpy()
            groundtruth_classes = objects['type'].numpy()+1
            vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          groundtruth_boxes,
          groundtruth_classes,
            None,
          cat_index,
          use_normalized_coordinates=True,
          line_thickness=1,
            groundtruth_box_visualization_color='blue')
        
    frame +=1
    return image_np

In [469]:
run_locnet( [[0.41085333,0.4335749 ,0.5223467 ,0.4712963 ]])

array([[ 0.41085333,  0.4335749 ,  0.5223467 ,  0.4712963 , -0.75153714,
         1.01438296, 28.55563545]])

In [470]:

# Load the TensorBoard notebook extension
%load_ext tensorboard

%tensorboard --logdir logs


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with -6).
Contents of stderr:
2020-08-21 21:30:34.280623: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "/opt/conda/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/opt/conda/lib/python3.7/site-packages/tensorboard/main.py", line 75, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/opt/conda/lib/python3.7/site-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/opt/conda/lib/python3.7/site-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "/opt/conda/lib/python3.7/site-packages/tensorboard/program.py", line 290, in main
    return runner(self.flags) or 0
  File "/opt/conda/lib/python3.7/site-packages/tensorboard/program.py", line 306, in _run_serve_subcommand
    server = self._make_server()
  File "/opt/conda/lib/python

In [471]:
def show_inference(model, tensor, objects, groundtruth=None, mode=None):
  
    image_np = np.array(tensor)
    image =process_image(image_np, objects, groundtruth, mode)
    display(Image.fromarray(image))
    return

Experiments

In [ ]:

for example in ds_test.take(5):  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    image = example["image"]
    objects = example["objects"]
  
    print('bbox:' ,objects['bbox'])
    print('location:', objects['location'])
    print('type:', objects['type'])
    show_inference(detection_model, image, objects, mode='locations')
    

In [473]:
print(len(ds_test))

711


In [474]:
import time

total = 0
for example in ds_test:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    image = example["image"]
    objects = example["objects"]
  
    start = time.time()
    image_np = np.array(image)
    run_inference_for_single_image(detection_model, image_np)
    end = time.time()
    total += (end - start)
    
print( total/len(ds_test))



0.07558770890645002


## Notes on bbox from tfds

bbox: tf.Tensor of type `tf.float32` and shape `[4,]` which contains the
      normalized coordinates of the bounding box `[ymin, xmin, ymax, xmax]`

In [475]:
cat_index

{1: {'id': 1, 'name': 'car'},
 2: {'id': 2, 'name': 'van'},
 3: {'id': 3, 'name': 'truck'},
 4: {'id': 4, 'name': 'pedestrian'}}

In [ ]:
#QUALITATIVE EVALUATION STEPS

from object_detection import eval_util
from object_detection.core import standard_fields as fields
from object_detection.metrics import coco_evaluation
from object_detection.protos import eval_pb2
from object_detection.utils import test_case
from object_detection.utils import tf_version

input_data_fields = fields.InputDataFields
detection_fields = fields.DetectionResultFields


for example in ds_test.take(5):  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    image = example["image"]
    objects = example["objects"]
#     show_inference(detection_model, image)
    image_np = np.array(image)
    # Actual detection.
    output_dict = run_inference_for_single_image(detection_model, image_np)


    output_dict = filter_detections( output_dict)

    batch_size = 1
    key=tf.constant('image1')
    
    groundtruth_boxes = format_boxes(objects['bbox'])
    groundtruth_classes = objects['type']+1
    groundtruth = {
        input_data_fields.groundtruth_boxes: groundtruth_boxes,
        input_data_fields.groundtruth_classes: groundtruth_classes,      
    }
    
    num_detections = tf.constant([len(output_dict['detection_classes'])])
    
    detections = {
        detection_fields.detection_boxes: tf.constant([output_dict['detection_boxes']]) ,
        detection_fields.detection_scores: tf.constant([output_dict['detection_scores']]),
        detection_fields.detection_classes: tf.constant([output_dict['detection_classes']-1]),
        detection_fields.num_detections: num_detections
       
    }
    

    image = tf.constant([image_np])
    
    
    result_dict = eval_util.result_dict_for_single_example(image, key,detections, groundtruth)
    
    side_by_side_img =  vis_util.draw_side_by_side_evaluation_image(result_dict,cat_index)[0][0].numpy()
    display(Image.fromarray(side_by_side_img))


In [478]:
#TODO: Complete Evaluation steps
from object_detection.core import standard_fields
from object_detection.utils import object_detection_evaluation


categories = [{
        'id': 1,
        'name': 'car'
    }, {
        'id': 4,
        'name': 'pedestrian'
    }]

pascal_evaluator = object_detection_evaluation.PascalDetectionEvaluator(
        categories,matching_iou_threshold=0.5)

for index, example in enumerate(ds_test):  
    image = example["image"]
    objects = example["objects"]
  
    image_np = np.array(image)
    # Actual detection.
    output_dict = run_inference_for_single_image(detection_model, image_np)
    output_dict = filter_detections( output_dict)
    
    image_key = 'img'+str(index)
    
    groundtruth_boxes = format_boxes(objects['bbox']).numpy()
    groundtruth_classes = objects['type'].numpy()+1


    pascal_evaluator.add_single_ground_truth_image_info(
        image_key,
        {standard_fields.InputDataFields.groundtruth_boxes: groundtruth_boxes,
         standard_fields.InputDataFields.groundtruth_classes:
         groundtruth_classes,
        })

    if len(output_dict['detection_classes']):
        pascal_evaluator.add_single_detected_image_info(
                image_key,
                {standard_fields.DetectionResultFields.detection_boxes: output_dict['detection_boxes'],
                 standard_fields.DetectionResultFields.detection_scores:
                 output_dict['detection_scores'],
                 standard_fields.DetectionResultFields.detection_classes:
                 output_dict['detection_classes']
                })
    

    metrics = pascal_evaluator.evaluate()
    

  num_images_correctly_detected_per_class / num_gt_imgs_per_class)



In [480]:
metrics

{'PascalBoxes_Precision/mAP@0.5IOU': 0.3248299516194694,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/car': 0.6589698941470521,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/pedestrian': 0.5965592185610505}

In [481]:
!pip install moviepy proglog

In [482]:
import tensorflow as tf
tf.test.gpu_device_name()

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [483]:
# !gsutil cp gs://hazard-detection-test-videos/accident_compilation.mp4 accident_compilation.mp4

In [484]:
!ls -sh accident_compilation.mp4

110M accident_compilation.mp4


In [485]:
from moviepy.editor import VideoFileClip
import proglog
proglog.notebook()

mot_tracker = Sort( max_age=8, iou_threshold=0.5)

tracked_ids=None
warning_ids = []
frame = 0
write_output = 'output_' + model_name + str(time.time())+ '.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first n seconds
# clip1 = VideoFileClip("test.mp4").subclip(0,1)

clip1 = VideoFileClip("accident_compilation.mp4")


white_clip = clip1.fl_image(process_image) 

white_clip.write_videofile(write_output, audio=False, verbose=False)

clip1.close()


Moviepy - Building video output_faster_rcnn_resnet50_lowproposals_coco_2018_01_281598045598.508329.mp4.
Moviepy - Writing video output_faster_rcnn_resnet50_lowproposals_coco_2018_01_281598045598.508329.mp4



KeyboardInterrupt: 